In [182]:
import openpyxl
import os
import glob
import numpy as np
import datetime
import pandas as pd

In [183]:
#os.chdir("/mydir")
files_list = glob.glob("*.xlsx")
files_list[:5]

['101_201712.xlsx',
 '1335_201712.xlsx',
 '1524_201712.xlsx',
 '3b48kzpzk8nbpghp-512245812_g1149_yield419.xlsx',
 '3b48kzpzk8nbs7yw-512245812_g1256_yield419.xlsx']

In [184]:
len(files_list)

572

# create empty dataframe

In [185]:
df = pd.DataFrame(columns=["file_name","fund_name","year","month","title","sub_title","value"])

# basic row structure

In [186]:
basic_row_struct = {"file_name":None,
                    "fund_name":None,
                   "year":None,
                   "month":None,
                   "title":None,
                   "sub_title":None,
                   "value":None}

# add file name to struct

In [187]:
def add_file_name_to_struct(file_name, basic_row_struct):
    basic_row_struct['file_name'] = file_name

# add_fund_name_and_get_worksheet

In [188]:
def add_fund_name_and_get_worksheet(workbook, basic_row_struct):
    fund_name = workbook.sheetnames[0] # there's only only one sheet in position 0
    basic_row_struct["fund_name"] = fund_name
    # get work sheet
    worksheet = workbook[fund_name]
    return worksheet

# add year to struct xlsx

In [195]:
def add_year(worksheet,basic_row_struct):
    if worksheet.cell(4,1).value != None and type(worksheet.cell(4,1).value) == int:
        year = worksheet.cell(4,1).value
    elif worksheet.cell(4,2).value != None and type(worksheet.cell(4,2).value) == int:
        year = worksheet.cell(4,2).value
    else:
        col1 = 7
        col2 = 8
        for row_num in range(1,17):
            val1 = worksheet.cell(row_num,col1).value
            val2 = worksheet.cell(row_num,col2).value
        
            if val1 != None:
                if type(val1) == datetime.datetime:
                    year = val1.year
                    break
                if type(val1) == int:
                    year = val1
                    break
                arr1 = [int(s) for s in val1.split() if s.isdigit()]
                if arr1:
                    year = arr1[0]
                    break
            if val2 != None:
                if type(val2) == datetime.datetime:
                    year = val2.year
                    break
                if type(val2) == int:
                    year = val2
                    break
                arr2 = [int(s) for s in val2.split() if s.isdigit()]
                if arr2:
                    year = arr2[0]
                    break

    if len(str(year)) > 4:
        year = int(str(year)[:-1])
    basic_row_struct['year'] = year

# find top corner xlsx:

In [190]:
def find_top_corner(worksheet):
    exit_loop = False
    for row in worksheet.iter_rows(max_row=18,max_col=4):
        for cell in row:
            if type(cell.value) == str and cell.value.find('מזומנים') != -1:
                start_row = cell.row
                start_col = cell.column
                #print(cell.value)
                exit_loop = True
                break
        if exit_loop == True:
            break
    return start_row, start_col

# find end row xlsx:

In [191]:
def find_end_row(worksheet):
    exit_loop = False
    for row in worksheet.iter_rows(min_row=25,max_row=37,max_col=4):
        for cell in row:
            if type(cell.value) == str and cell.value.find('רווח') != -1 :
                end_row = cell.row
                #print(cell.value)
                exit_loop = True
                break
        if exit_loop == True:
            break
    return end_row

# iterate table and get values xlsx:

In [192]:
def get_values_and_add_to_df(worksheet, start_row, end_row, start_col, basic_row_struct):
    df = pd.DataFrame(columns=["file_name","fund_name","year","month","title","sub_title","value"])
    for row in worksheet.iter_rows(min_row=start_row, max_row=end_row, min_col=start_col, max_col=start_col+24):
        i = 0
        month = 1
        for cell in row:
            if i == 0:
                basic_row_struct['title'] = cell.value
            elif i%2 == 1:
                basic_row_struct['sub_title'] = 'תרומה לתשואה'
                basic_row_struct['month'] = month
                basic_row_struct['value'] = cell.value
            else:
                basic_row_struct['sub_title'] = 'שיעור מסך הנכסים'
                basic_row_struct['month'] = month
                basic_row_struct['value'] = cell.value
                month+=1
            if i!= 0:
                df = df.append(basic_row_struct,ignore_index=True)
            i+=1
    return df

# go over all xlsx files

In [193]:
for i in range(len(files_list[:])):
    file_name = files_list[i]
    filename_no_extention, extension = os.path.splitext(file_name)
    workbook = openpyxl.load_workbook(filename=file_name)
    add_file_name_to_struct(file_name,basic_row_struct)
    worksheet = add_fund_name_and_get_worksheet(workbook,basic_row_struct)
    add_year(worksheet, basic_row_struct)
    start_row, start_col = find_top_corner(worksheet)
    end_row = find_end_row(worksheet)
    df = get_values_and_add_to_df(worksheet,start_row,end_row,start_col,basic_row_struct)
    path = 'output_xlsx/' + filename_no_extention + '.csv'
    df.to_csv(path,index = False, encoding = 'utf-8-sig')

C:\Users\orenz\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:214: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


AttributeError: 'int' object has no attribute 'split'

In [194]:
file_name

'דוח-תשואה-לאפיקים-כללית-60-פלוס רבעון 4 2017.xlsx'